# 01 — QC & Preprocessing (Systemic Sclerosis Skin)

Loads **GSE138669** (multiple `.h5`), performs QC, merges samples, and writes `data/processed/ssc_skin_qc.h5ad`.

In [ ]:
# %pip install scanpy anndata scvi-tools scrublet GEOparse  # if not using conda

## 1) Configure paths

In [ ]:
from pathlib import Path
RAW=Path('data/raw'); PROC=Path('data/processed'); RAW.mkdir(parents=True,exist_ok=True); PROC.mkdir(parents=True,exist_ok=True)
RAW.resolve(), PROC.resolve()

## 2) (Optional) Download GSE138669 via GEOparse

In [ ]:
!python src/data_download.py --geo GSE138669

## 3) Load & merge data from `data/raw/`

In [ ]:
import scanpy as sc, anndata as ad, numpy as np, re
from pathlib import Path
def load_any_merge(path: Path):
    h5s = sorted(path.glob("**/*.h5"))
    if h5s:
        adatas = []
        def sample_name(p):
            import re
            m = re.match(r"(GSM\d+)", p.stem)
            return m.group(1) if m else p.stem
        for p in h5s:
            print(f"[info] loading 10x HDF5: {p}")
            ad = sc.read_10x_h5(p)
            ad.obs["sample"] = sample_name(p)
            adatas.append(ad)
        if len(adatas) == 1:
            return adatas[0]
        return adatas[0].concatenate(*adatas[1:], batch_key="sample", batch_categories=[a.obs["sample"][0] for a in adatas])
    h5ads = sorted(path.glob("**/*.h5ad"))
    if len(h5ads) > 1:
        adatas = []
        for p in h5ads:
            ad = sc.read_h5ad(p)
            ad.obs["sample"] = p.stem
            adatas.append(ad)
        return adatas[0].concatenate(*adatas[1:], batch_key="sample", batch_categories=[a.obs["sample"][0] for a in adatas])
    elif len(h5ads) == 1:
        return sc.read_h5ad(h5ads[0])
    mtx_dirs = [p for p in path.glob("**/*") if p.is_dir() and (p/'matrix.mtx').exists()]
    if mtx_dirs:
        return sc.read_10x_mtx(mtx_dirs[0], var_names='gene_symbols', cache=True)
    raise FileNotFoundError("No supported data found. Place .h5, .h5ad, or a 10x mtx folder under data/raw/.")
adata = load_any_merge(RAW); adata

## 3a) Discover `.h5` samples and pre-QC counts

In [ ]:
from pathlib import Path
import pandas as pd, scanpy as sc, re
RAW = Path('data/raw')
def sample_name(p: Path) -> str:
    m = re.match(r'(GSM\d+)', p.stem)
    return m.group(1) if m else p.stem
h5s = sorted(RAW.glob('**/*.h5'))
print(f'Found {len(h5s)} HDF5 files')
display(pd.DataFrame({'file': [str(p) for p in h5s], 'sample': [sample_name(p) for p in h5s]}))
pre_counts = []
for p in h5s:
    try:
        ad = sc.read_10x_h5(p)
        pre_counts.append({'sample': sample_name(p), 'cells_pre_qc': ad.n_obs})
    except Exception as e:
        pre_counts.append({'sample': sample_name(p), 'cells_pre_qc': None, 'error': str(e)})
pre_df = pd.DataFrame(pre_counts).set_index('sample'); display(pre_df)

## 4) Basic QC

In [ ]:
adata.var['mt']=adata.var_names.str.upper().str.startswith(('MT-','MT.'))
sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'],percent_top=None,inplace=True)
keep=(adata.obs['n_genes_by_counts']>=200)&(adata.obs['pct_counts_mt']<=10)
adata=adata[keep].copy()
sc.pp.filter_genes(adata,min_cells=3)
adata.shape

## 5) Normalize, log1p, HVGs

In [ ]:
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=4000,flavor='seurat_v3')
hvg = adata[:, adata.var['highly_variable']].copy(); hvg

## 6) Save + Post-QC per-sample counts

In [ ]:
out = PROC/'ssc_skin_qc.h5ad'
adata.write(out)
print('[done] wrote', out.resolve())
import pandas as pd
if 'sample' in adata.obs.columns:
    post_df = adata.obs['sample'].value_counts().rename('cells_post_qc').to_frame()
    display(post_df.sort_index())
else:
    print("[note] 'sample' not present; per-sample counts unavailable.")